In [4]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option("display.max_columns", None)
sns.set_style("whitegrid")

In [5]:
df_raw = pd.read_csv("Lead Scoring.csv")
df_raw.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,NaN,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Interested in other courses,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,NaN,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


In [6]:
#((df_raw['Total Time Spent on Website'].describe()/60).round(2))

In [7]:
df_raw.shape

(9240, 37)

In [8]:
df_raw.columns

Index(['Prospect ID', 'Lead Number', 'Lead Origin', 'Lead Source',
       'Do Not Email', 'Do Not Call', 'Converted', 'TotalVisits',
       'Total Time Spent on Website', 'Page Views Per Visit', 'Last Activity',
       'Country', 'Specialization', 'How did you hear about X Education',
       'What is your current occupation',
       'What matters most to you in choosing a course', 'Search', 'Magazine',
       'Newspaper Article', 'X Education Forums', 'Newspaper',
       'Digital Advertisement', 'Through Recommendations',
       'Receive More Updates About Our Courses', 'Tags', 'Lead Quality',
       'Update me on Supply Chain Content', 'Get updates on DM Content',
       'Lead Profile', 'City', 'Asymmetrique Activity Index',
       'Asymmetrique Profile Index', 'Asymmetrique Activity Score',
       'Asymmetrique Profile Score',
       'I agree to pay the amount through cheque',
       'A free copy of Mastering The Interview', 'Last Notable Activity'],
      dtype='object')

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

In [10]:
df_raw.isnull().sum()

Prospect ID                                         0
Lead Number                                         0
Lead Origin                                         0
Lead Source                                        36
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                       137
Total Time Spent on Website                         0
Page Views Per Visit                              137
Last Activity                                     103
Country                                          2461
Specialization                                   1438
How did you hear about X Education               2207
What is your current occupation                  2690
What matters most to you in choosing a course    2709
Search                                              0
Magazine                                            0
Newspaper Article           

In [11]:
missing_percentage =  df_raw.isnull().sum() * 100 / len(df_raw)
missing_percentage

Prospect ID                                       0.000000
Lead Number                                       0.000000
Lead Origin                                       0.000000
Lead Source                                       0.389610
Do Not Email                                      0.000000
Do Not Call                                       0.000000
Converted                                         0.000000
TotalVisits                                       1.482684
Total Time Spent on Website                       0.000000
Page Views Per Visit                              1.482684
Last Activity                                     1.114719
Country                                          26.634199
Specialization                                   15.562771
How did you hear about X Education               23.885281
What is your current occupation                  29.112554
What matters most to you in choosing a course    29.318182
Search                                            0.0000

In [12]:
df_raw.describe()

,Lead Number,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score
count,9240.000000,9240.000000,9103.000000,9240.000000,9103.000000,5022.000000,5022.000000
mean,617188.435606,0.385390,3.445238,487.698268,2.362820,14.306252,16.344883
std,23405.995698,0.486714,4.854853,548.021466,2.161418,1.386694,1.811395
min,579533.000000,0.000000,0.000000,0.000000,0.000000,7.000000,11.000000
25%,596484.500000,0.000000,1.000000,12.000000,1.000000,14.000000,15.000000
50%,615479.000000,0.000000,3.000000,248.000000,2.000000,14.000000,16.000000
75%,637387.250000,1.000000,5.000000,936.000000,3.000000,15.000000,18.000000
max,660737.000000,1.000000,251.000000,2272.000000,55.000000,18.000000,20.000000


In [13]:
#copy the raw data for cleaning
df=df_raw.copy()
df.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,NaN,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Interested in other courses,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,NaN,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


In [14]:
df.drop(columns=["Prospect ID", "Lead Number"], inplace=True)
df.columns

Index(['Lead Origin', 'Lead Source', 'Do Not Email', 'Do Not Call',
       'Converted', 'TotalVisits', 'Total Time Spent on Website',
       'Page Views Per Visit', 'Last Activity', 'Country', 'Specialization',
       'How did you hear about X Education', 'What is your current occupation',
       'What matters most to you in choosing a course', 'Search', 'Magazine',
       'Newspaper Article', 'X Education Forums', 'Newspaper',
       'Digital Advertisement', 'Through Recommendations',
       'Receive More Updates About Our Courses', 'Tags', 'Lead Quality',
       'Update me on Supply Chain Content', 'Get updates on DM Content',
       'Lead Profile', 'City', 'Asymmetrique Activity Index',
       'Asymmetrique Profile Index', 'Asymmetrique Activity Score',
       'Asymmetrique Profile Score',
       'I agree to pay the amount through cheque',
       'A free copy of Mastering The Interview', 'Last Notable Activity'],
      dtype='object')

In [15]:
df.describe()

,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score
count,9240.000000,9103.000000,9240.000000,9103.000000,5022.000000,5022.000000
mean,0.385390,3.445238,487.698268,2.362820,14.306252,16.344883
std,0.486714,4.854853,548.021466,2.161418,1.386694,1.811395
min,0.000000,0.000000,0.000000,0.000000,7.000000,11.000000
25%,0.000000,1.000000,12.000000,1.000000,14.000000,15.000000
50%,0.000000,3.000000,248.000000,2.000000,14.000000,16.000000
75%,1.000000,5.000000,936.000000,3.000000,15.000000,18.000000
max,1.000000,251.000000,2272.000000,55.000000,18.000000,20.000000


In [16]:
# yes_no_columns = []

# for col in df.columns:
#     values = df[col].astype(str).str.lower().unique()
    
#     if 'yes' in values or 'no' in values:
#         yes_no_columns.append(col)

# yes_no_columns

In [17]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing

Lead Origin                                       0.000000
Lead Source                                       0.389610
Do Not Email                                      0.000000
Do Not Call                                       0.000000
Converted                                         0.000000
TotalVisits                                       1.482684
Total Time Spent on Website                       0.000000
Page Views Per Visit                              1.482684
Last Activity                                     1.114719
Country                                          26.634199
Specialization                                   15.562771
How did you hear about X Education               23.885281
What is your current occupation                  29.112554
What matters most to you in choosing a course    29.318182
Search                                            0.000000
Magazine                                          0.000000
Newspaper Article                                 0.0000

In [18]:
df["Lead Source"].fillna(df["Lead Source"].mode()[0], inplace=True)

top_sources = ["Google", "Direct Traffic", "Organic Search", "Facebook", "Olark Chat"]

df["Lead Source"] = df["Lead Source"].apply(
    lambda x: x if x in top_sources else "Other"
)

C:\Users\aftab\AppData\Local\Temp\ipykernel_18304\2852760247.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Lead Source"].fillna(df["Lead Source"].mode()[0], inplace=True)


In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
df['Lead Source'].unique()

array(['Olark Chat', 'Organic Search', 'Direct Traffic', 'Google',
       'Other', 'Facebook'], dtype=object)

In [21]:
df.isnull().sum()

Lead Origin                                         0
Lead Source                                         0
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                       137
Total Time Spent on Website                         0
Page Views Per Visit                              137
Last Activity                                     103
Country                                          2461
Specialization                                   1438
How did you hear about X Education               2207
What is your current occupation                  2690
What matters most to you in choosing a course    2709
Search                                              0
Magazine                                            0
Newspaper Article                                   0
X Education Forums                                  0
Newspaper                   

In [22]:
percent_missing

Lead Origin                                       0.000000
Lead Source                                       0.389610
Do Not Email                                      0.000000
Do Not Call                                       0.000000
Converted                                         0.000000
TotalVisits                                       1.482684
Total Time Spent on Website                       0.000000
Page Views Per Visit                              1.482684
Last Activity                                     1.114719
Country                                          26.634199
Specialization                                   15.562771
How did you hear about X Education               23.885281
What is your current occupation                  29.112554
What matters most to you in choosing a course    29.318182
Search                                            0.000000
Magazine                                          0.000000
Newspaper Article                                 0.0000

In [23]:
#Reason outliers present in the columns
df["TotalVisits"] = df["TotalVisits"].fillna(df["TotalVisits"].median())

In [24]:
df['Page Views Per Visit']=df["Page Views Per Visit"].fillna(df["Page Views Per Visit"].median())

In [25]:
df.isnull().sum()

Lead Origin                                         0
Lead Source                                         0
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                         0
Total Time Spent on Website                         0
Page Views Per Visit                                0
Last Activity                                     103
Country                                          2461
Specialization                                   1438
How did you hear about X Education               2207
What is your current occupation                  2690
What matters most to you in choosing a course    2709
Search                                              0
Magazine                                            0
Newspaper Article                                   0
X Education Forums                                  0
Newspaper                   

In [26]:
df['Last Activity'].unique()

array(['Page Visited on Website', 'Email Opened', 'Unreachable',
       'Converted to Lead', 'Olark Chat Conversation', 'Email Bounced',
       'Email Link Clicked', 'Form Submitted on Website', 'Unsubscribed',
       'Had a Phone Conversation', 'View in browser link Clicked', nan,
       'Approached upfront', 'SMS Sent', 'Visited Booth in Tradeshow',
       'Resubscribed to emails', 'Email Received', 'Email Marked Spam'],
      dtype=object)

In [27]:
df['Last Activity']=df['Last Activity'].fillna(df['Last Activity'].mode()[0])

In [28]:
df['Last Activity'].unique()

array(['Page Visited on Website', 'Email Opened', 'Unreachable',
       'Converted to Lead', 'Olark Chat Conversation', 'Email Bounced',
       'Email Link Clicked', 'Form Submitted on Website', 'Unsubscribed',
       'Had a Phone Conversation', 'View in browser link Clicked',
       'Approached upfront', 'SMS Sent', 'Visited Booth in Tradeshow',
       'Resubscribed to emails', 'Email Received', 'Email Marked Spam'],
      dtype=object)

In [29]:
df['Last Activity'].isnull().sum()

np.int64(0)

In [30]:
df.isnull().sum()

Lead Origin                                         0
Lead Source                                         0
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                         0
Total Time Spent on Website                         0
Page Views Per Visit                                0
Last Activity                                       0
Country                                          2461
Specialization                                   1438
How did you hear about X Education               2207
What is your current occupation                  2690
What matters most to you in choosing a course    2709
Search                                              0
Magazine                                            0
Newspaper Article                                   0
X Education Forums                                  0
Newspaper                   

In [31]:
percent_missing

Lead Origin                                       0.000000
Lead Source                                       0.389610
Do Not Email                                      0.000000
Do Not Call                                       0.000000
Converted                                         0.000000
TotalVisits                                       1.482684
Total Time Spent on Website                       0.000000
Page Views Per Visit                              1.482684
Last Activity                                     1.114719
Country                                          26.634199
Specialization                                   15.562771
How did you hear about X Education               23.885281
What is your current occupation                  29.112554
What matters most to you in choosing a course    29.318182
Search                                            0.000000
Magazine                                          0.000000
Newspaper Article                                 0.0000

In [32]:
profile_cols = [
    "Specialization",
    "How did you hear about X Education",
    "What is your current occupation",
    "What matters most to you in choosing a course"
]

for col in profile_cols:
    df[col].fillna("Unknown", inplace=True)

In [33]:
df_cat= df[[
    "Specialization",
    "How did you hear about X Education",
    "What is your current occupation",
    "What matters most to you in choosing a course"
]]

data=pd.DataFrame(df_cat)
data

,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course
0,Select,Select,Unemployed,Better Career Prospects
1,Select,Select,Unemployed,Better Career Prospects
2,Business Administration,Select,Student,Better Career Prospects
3,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects
4,Select,Other,Unemployed,Better Career Prospects
...,...,...,...,...
9235,IT Projects Management,Select,Unemployed,Better Career Prospects
9236,Media and Advertising,Select,Unemployed,Better Career Prospects
9237,Business Administration,Select,Unemployed,Better Career Prospects
9238,Human Resource Management,Online Search,Unknown,Unknown


In [34]:
percent_missing

Lead Origin                                       0.000000
Lead Source                                       0.389610
Do Not Email                                      0.000000
Do Not Call                                       0.000000
Converted                                         0.000000
TotalVisits                                       1.482684
Total Time Spent on Website                       0.000000
Page Views Per Visit                              1.482684
Last Activity                                     1.114719
Country                                          26.634199
Specialization                                   15.562771
How did you hear about X Education               23.885281
What is your current occupation                  29.112554
What matters most to you in choosing a course    29.318182
Search                                            0.000000
Magazine                                          0.000000
Newspaper Article                                 0.0000

In [35]:
df["Country"]=df["Country"].fillna("Unknown")

In [36]:
df["Country"].unique()

array(['Unknown', 'India', 'Russia', 'Kuwait', 'Oman',
       'United Arab Emirates', 'United States', 'Australia',
       'United Kingdom', 'Bahrain', 'Ghana', 'Singapore', 'Qatar',
       'Saudi Arabia', 'Belgium', 'France', 'Sri Lanka', 'China',
       'Canada', 'Netherlands', 'Sweden', 'Nigeria', 'Hong Kong',
       'Germany', 'Asia/Pacific Region', 'Uganda', 'Kenya', 'Italy',
       'South Africa', 'Tanzania', 'unknown', 'Malaysia', 'Liberia',
       'Switzerland', 'Denmark', 'Philippines', 'Bangladesh', 'Vietnam',
       'Indonesia'], dtype=object)

In [37]:
df.isnull().sum()

Lead Origin                                         0
Lead Source                                         0
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                         0
Total Time Spent on Website                         0
Page Views Per Visit                                0
Last Activity                                       0
Country                                             0
Specialization                                      0
How did you hear about X Education                  0
What is your current occupation                     0
What matters most to you in choosing a course       0
Search                                              0
Magazine                                            0
Newspaper Article                                   0
X Education Forums                                  0
Newspaper                   

In [38]:
binary_cols = [
    "Do Not Email", "Do Not Call", "Search", "Magazine",
    "Newspaper Article", "X Education Forums", "Newspaper",
    "Digital Advertisement", "Through Recommendations",
    "Receive More Updates About Our Courses",
    "Update me on Supply Chain Content", "Get updates on DM Content",
    "I agree to pay the amount through cheque",
    "A free copy of Mastering The Interview",
    "Lead Quality"
]

for col in binary_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .map({"yes": 1, "no": 0})
        .fillna(0)   # 👈 future-proof line
    )

In [36]:
df.isnull().sum()

Lead Origin                                         0
Lead Source                                         0
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                         0
Total Time Spent on Website                         0
Page Views Per Visit                                0
Last Activity                                       0
Country                                             0
Specialization                                      0
How did you hear about X Education                  0
What is your current occupation                     0
What matters most to you in choosing a course       0
Search                                              0
Magazine                                            0
Newspaper Article                                   0
X Education Forums                                  0
Newspaper                   

In [37]:
for col in binary_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .apply(lambda x: 1 if x == "yes" else 0)
    )

In [39]:
def tag_mapper(tag):
    if pd.isna(tag):
        return "Unknown"
    elif "Interested" in tag:
        return "High Intent"
    elif tag in ["Busy", "Ringing"]:
        return "Medium Intent"
    else:
        return "Low Intent"

df["Tag_Group"] = df["Tags"].apply(tag_mapper)
df.drop(columns=["Tags"], inplace=True)


In [40]:
df['Tag_Group']

0         High Intent
1       Medium Intent
2          Low Intent
3       Medium Intent
4          Low Intent
            ...      
9235       Low Intent
9236       Low Intent
9237       Low Intent
9238          Unknown
9239       Low Intent
Name: Tag_Group, Length: 9240, dtype: object

In [41]:
df["Lead Quality"]=df["Lead Quality"].fillna(df["Lead Quality"].mode()[0])

In [42]:
df["Lead Quality"].unique()

array([0.])

In [43]:
# df.drop(columns=["Lead Quality"], inplace=True)

In [44]:
df.head()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity,Tag_Group
0,API,Olark Chat,0,0,0,0.0,0,0.0,Page Visited on Website,Unknown,Select,Select,Unemployed,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Select,Select,02.Medium,02.Medium,15.0,15.0,0,0,Modified,High Intent
1,API,Organic Search,0,0,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Select,Select,02.Medium,02.Medium,15.0,15.0,0,0,Email Opened,Medium Intent
2,Landing Page Submission,Direct Traffic,0,0,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,0,1,Email Opened,Low Intent
3,Landing Page Submission,Direct Traffic,0,0,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Select,Mumbai,02.Medium,01.High,13.0,17.0,0,0,Modified,Medium Intent
4,Landing Page Submission,Google,0,0,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Select,Mumbai,02.Medium,01.High,15.0,18.0,0,0,Modified,Low Intent


In [45]:
df.columns

Index(['Lead Origin', 'Lead Source', 'Do Not Email', 'Do Not Call',
       'Converted', 'TotalVisits', 'Total Time Spent on Website',
       'Page Views Per Visit', 'Last Activity', 'Country', 'Specialization',
       'How did you hear about X Education', 'What is your current occupation',
       'What matters most to you in choosing a course', 'Search', 'Magazine',
       'Newspaper Article', 'X Education Forums', 'Newspaper',
       'Digital Advertisement', 'Through Recommendations',
       'Receive More Updates About Our Courses', 'Lead Quality',
       'Update me on Supply Chain Content', 'Get updates on DM Content',
       'Lead Profile', 'City', 'Asymmetrique Activity Index',
       'Asymmetrique Profile Index', 'Asymmetrique Activity Score',
       'Asymmetrique Profile Score',
       'I agree to pay the amount through cheque',
       'A free copy of Mastering The Interview', 'Last Notable Activity',
       'Tag_Group'],
      dtype='object')

In [46]:
df.shape[1]

35

In [47]:
df['Lead Profile'].unique()

array(['Select', 'Potential Lead', nan, 'Other Leads', 'Lateral Student',
       'Dual Specialization Student', 'Student of SomeSchool'],
      dtype=object)

In [48]:
df["Lead Profile"].fillna("Unknown", inplace=True)

In [49]:
df['Lead Profile'].isnull().sum()

np.int64(0)

In [50]:
df.drop(columns=[
    "Asymmetrique Activity Index",
    "Asymmetrique Profile Index"
], inplace=True)

In [51]:
df.shape[1]

33

In [52]:
df["Asymmetrique Activity Score"].fillna(
    df["Asymmetrique Activity Score"].median(), inplace=True
)

In [53]:
df["Asymmetrique Profile Score"].fillna(
    df["Asymmetrique Profile Score"].median(), inplace=True
)

In [54]:
df.isnull().sum()

Lead Origin                                         0
Lead Source                                         0
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                         0
Total Time Spent on Website                         0
Page Views Per Visit                                0
Last Activity                                       0
Country                                             0
Specialization                                      0
How did you hear about X Education                  0
What is your current occupation                     0
What matters most to you in choosing a course       0
Search                                              0
Magazine                                            0
Newspaper Article                                   0
X Education Forums                                  0
Newspaper                   

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 33 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Lead Origin                                    9240 non-null   object 
 1   Lead Source                                    9240 non-null   object 
 2   Do Not Email                                   9240 non-null   int64  
 3   Do Not Call                                    9240 non-null   int64  
 4   Converted                                      9240 non-null   int64  
 5   TotalVisits                                    9240 non-null   float64
 6   Total Time Spent on Website                    9240 non-null   int64  
 7   Page Views Per Visit                           9240 non-null   float64
 8   Last Activity                                  9240 non-null   object 
 9   Country                                        9240 

In [56]:
df["Total Time Spent on Website"] = df["Total Time Spent on Website"].astype(float)

In [57]:
df["Converted"].unique()

array([0, 1])

In [58]:
df.shape[1]

33

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
X = df.drop("Converted", axis=1)
y = df["Converted"]

In [61]:
X.head()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity,Tag_Group
0,API,Olark Chat,0,0,0.0,0.0,0.0,Page Visited on Website,Unknown,Select,Select,Unemployed,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Select,Select,15.0,15.0,0,0,Modified,High Intent
1,API,Organic Search,0,0,5.0,674.0,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Select,Select,15.0,15.0,0,0,Email Opened,Medium Intent
2,Landing Page Submission,Direct Traffic,0,0,2.0,1532.0,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Potential Lead,Mumbai,14.0,20.0,0,1,Email Opened,Low Intent
3,Landing Page Submission,Direct Traffic,0,0,1.0,305.0,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Select,Mumbai,13.0,17.0,0,0,Modified,Medium Intent
4,Landing Page Submission,Google,0,0,2.0,1428.0,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,0,0,0,0,0,0,0,0,0.0,0,0,Select,Mumbai,15.0,18.0,0,0,Modified,Low Intent


In [62]:
X.shape[1]

32

In [63]:
y.head()

0    0
1    0
2    1
3    0
4    1
Name: Converted, dtype: int64

In [64]:
#Preprocessing
categorical_cols = X.select_dtypes(include="object").columns
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns

categorical_cols

Index(['Lead Origin', 'Lead Source', 'Last Activity', 'Country',
       'Specialization', 'How did you hear about X Education',
       'What is your current occupation',
       'What matters most to you in choosing a course', 'Lead Profile', 'City',
       'Last Notable Activity', 'Tag_Group'],
      dtype='object')

In [65]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [75]:
categorical_transformer = OneHotEncoder(
    drop="first",
    handle_unknown="ignore")


In [76]:
categorical_transformer

,categories,'auto'
,drop,'first'
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [77]:
numeric_transformer = StandardScaler()

In [78]:
numeric_transformer

,copy,True
,with_mean,True
,with_std,True


In [79]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numeric_cols)
    ]
)


In [80]:
preprocessor

,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,'first'
,sparse_output,True


In [81]:
from sklearn.linear_model import LogisticRegression

In [82]:
lor_model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

In [83]:
lor_model

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [84]:
from sklearn.model_selection import train_test_split

In [85]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [86]:
from sklearn.pipeline import Pipeline

In [ ]:
clf_lor = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", lor_model)
    ]
)

In [ ]:
clf_lor

In [ ]:
clf_lor.fit(X_train, y_train)

In [ ]:
y_pred_lor = clf_lor.predict(X_test)

In [ ]:
y_pred_lor

In [ ]:
lor_comparison = pd.DataFrame({
    "Actual": y_test,
    "Predicted": y_pred_lor
})

lor_comparison.head()

In [ ]:
(lor_comparison["Actual"] == lor_comparison["Predicted"]).value_counts()

In [ ]:
((lor_comparison["Actual"] == lor_comparison["Predicted"]).value_counts(normalize=True) * 100).round(2)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred_lor)
cm

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred_lor))

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
y_prob_lor = clf_lor.predict_proba(X_test)[:, 1] #probability of conversion[1]

In [ ]:
y_prob_lor

In [ ]:
roc_auc = roc_auc_score(y_test, y_prob_lor)
roc_auc

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        random_state=42
    ),

    "Decision Tree": DecisionTreeClassifier(
        max_depth=6,
        random_state=42
    ),

    "Random Forest": RandomForestClassifier(
        n_estimators=200,
        max_depth=8,
        random_state=42
    ),

    "Gradient Boosting": GradientBoostingClassifier(
        random_state=42
    ),

    "KNN": KNeighborsClassifier(
        n_neighbors=7
    ),

    "SVM": SVC(
        probability=True,
        kernel="rbf",
        random_state=42
    )
}

In [ ]:
results = []

for name, model in models.items():
    
    clf = Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("model", model)
    ])
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)[:, 1]
    
    roc_auc = roc_auc_score(y_test, y_prob)
    
    print(f"\n{name}")
    print("-" * 30)
    print("ROC-AUC:", round(roc_auc, 3))
    print(classification_report(y_test, y_pred))
    
    results.append({
        "Model": name,
        "ROC-AUC": roc_auc
    })


In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

### Cross validation with more accuracy given models

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

In [ ]:
log_reg = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

logreg_scores = cross_val_score(
    log_reg,
    X,
    y,
    cv=cv,
    scoring="roc_auc"
)

print("Logistic Regression CV ROC-AUC scores:", logreg_scores)
print("Mean ROC-AUC:", logreg_scores.mean())
print("Std Dev:", logreg_scores.std())

In [ ]:
gb_model = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", GradientBoostingClassifier(random_state=42))
])

gb_scores = cross_val_score(
    gb_model,
    X,
    y,
    cv=cv,
    scoring="roc_auc"
)

print("Gradient Boosting CV ROC-AUC scores:", gb_scores)
print("Mean ROC-AUC:", gb_scores.mean())
print("Std Dev:", gb_scores.std())

In [ ]:
svm_model = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", SVC(probability=True, random_state=42))
])

svm_scores = cross_val_score(
    svm_model,
    X,
    y,
    cv=cv,
    scoring="roc_auc"
)

print("SVM CV ROC-AUC scores:", svm_scores)
print("Mean ROC-AUC:", svm_scores.mean())
print("Std Dev:", svm_scores.std())


In [ ]:
cv_results = pd.DataFrame({
    "Model": ["Logistic Regression", "Gradient Boosting", "SVM"],
    "Mean ROC-AUC": [
        logreg_scores.mean(),
        gb_scores.mean(),
        svm_scores.mean()
    ],
    "Std Dev": [
        logreg_scores.std(),
        gb_scores.std(),
        svm_scores.std()
    ]
})

cv_results

### Naive Baise
### GaussianNB

In [ ]:
#Dense Data(All values stored)
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False), categorical_cols),
        ("num", StandardScaler(), numeric_cols)
    ]
)

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_gnb = GaussianNB()

In [ ]:
clf_gnb = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", model_gnb)
    ]
)

In [ ]:
clf_gnb.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score

y_prob = clf_gnb.predict_proba(X_test)[:, 1]

print("GaussianNB ROC-AUC:", roc_auc_score(y_test, y_prob))

### MultinominalNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model_mnb = MultinomialNB(alpha=1.0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols),
        ("num", MinMaxScaler(), numeric_cols)
    ]
)

In [ ]:
clf_mnb = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", model_mnb)
    ]
)

In [ ]:
clf_mnb.fit(X_train, y_train)

In [ ]:
y_prob = clf_mnb.predict_proba(X_test)[:,1]
print("Multinominal ROC-AUC:", roc_auc_score(y_test,y_prob))

### BernoulliNB

In [ ]:
from sklearn.naive_bayes import BernoulliNB

model_bnb = BernoulliNB(alpha=1.0)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False), categorical_cols),
        ("num", StandardScaler(), numeric_cols)
    ]
)

In [ ]:
clf_bnb = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", model_bnb)
    ]
)

In [ ]:
clf_bnb.fit(X_train,y_train)

In [ ]:
y_prob = clf_bnb.predict_proba(X_test)[:,1]
print("BernoulliNB:", roc_auc_score(y_test,y_prob))

### CatBoostClassifier

In [ ]:
#!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

model_cat = CatBoostClassifier(
    iterations=300,
    learning_rate=0.05,
    depth=6,
    loss_function="Logloss",
    eval_metric="AUC",
    auto_class_weights="Balanced",
    random_state=42,
    verbose=0
)

In [ ]:
clf_catb = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", model_cat)
    ]
)

In [ ]:
clf_catb.fit(X_train,y_train)

In [ ]:
y_prob = clf_catb.predict_proba(X_test)[:,1]

print("CatBoostClassifier:", roc_auc_score(y_test,y_prob))

### LightGBM

In [ ]:
#!pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

model_lgbm = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight="balanced",
    random_state=42
)

In [ ]:
categorical_cols

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numeric_cols)
    ]
)

In [ ]:
clf_lgbm = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", model_lgbm)
    ]
)

In [ ]:
clf_lgbm.fit(X_train,y_train)

In [ ]:
y_prob = clf_lgbm.predict_proba(X_test)[:,1]
print("LightGBM ROC-AUC:", roc_auc_score(y_test,y_prob))

In [ ]:
X_train

In [ ]:
#conda install -c conda-forge xgboost

### XGBoost

In [ ]:
import xgboost
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False), categorical_cols),
        ("num", StandardScaler(), numeric_cols)
    ]
)

In [ ]:
from sklearn.model_selection import cross_val_predict

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scale_pos_weight = (y == 0).sum() / (y == 1).sum()

xgb_clf = Pipeline([
    ("preprocessing", preprocessor),
    ("model", XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",
        random_state=42
    ))
])

xgb_clf.fit(X_train,y_train)

xgb_clf_prob=xgb_clf.predict_proba(X_test)[:,1]

print("XGB ROC AUC:",roc_auc_score(y_test,xgb_clf_prob))

# Get CV probabilities
y_cv_prob = cross_val_predict(
    xgb_clf,
    X,
    y,
    cv=cv,
    method="predict_proba"
)[:, 1]

# Computed ROC-AUC manually
roc_auc = roc_auc_score(y, y_cv_prob)#completely unseen data
print("XGBoost CV ROC-AUC:", roc_auc)#out-of-fold predictions from cross-validation

In [ ]:
#len(X.columns)#==31

In [ ]:
# y_prob = clf.predict_proba(X_test)[:, 1]
# y_prob

In [ ]:
# from sklearn.metrics import auc,roc_curve, roc_auc_score

In [ ]:
# fpr, tpr, thresholds = roc_curve(y_test, y_prob)
# roc_auc = auc(fpr, tpr)
# roc_auc

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
# plt.figure(figsize=(7, 5))

# plt.plot(
#     fpr,
#     tpr,
#     label=f"ROC Curve (AUC = {roc_auc:.3f})"
# )

# # Random model reference line
# plt.plot([0, 1], [0, 1], linestyle="--", label="Random Model")

# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("ROC Curve - Gradient Boosting")
# plt.legend(loc="lower right")
# plt.grid(True)
# plt.show()

In [ ]:
# # Train probabilities
# y_train_prob = clf.predict_proba(X_train)[:, 1]

# fpr_train, tpr_train, _ = roc_curve(y_train, y_train_prob)
# roc_auc_train = auc(fpr_train, tpr_train)

# plt.figure(figsize=(7,5))
# plt.plot(fpr_train, tpr_train, label=f"Train AUC = {roc_auc_train:.3f}")
# plt.plot(fpr, tpr, label=f"Test AUC = {roc_auc:.3f}")
# plt.plot([0,1],[0,1],'--')
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("Train vs Test ROC Curve")
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
# def lead_bucket(score):
#     if score >= 80:
#         return "Hot Lead"
#     elif score >= 60:
#         return "Warm Lead"
#     else:
#         return "Cold Lead"

# lead_category = pd.Series(lead_score).apply(lead_bucket)
# lead_category.head()